Tämä tiedosto lukee Microsoft CDM zip:n (https://github.com/microsoft/CDM) eri tarpeisiin (zip-tiedosto on ladattu GitHubista 17.12.2020). Jos ja kun on tarve jatkoanalysoida json-tiedostoja esimerkiksi Power BI Desktop -työkalulla, niin sekä potentiaalisia entiteettejä (cdm.json-loppuiset tiedostot, joihin ei sisälly manifest.cdm.json-tiedostot) ja niiden suhteista (manifest.cdm.json-loppuiset tiedostot) kuvaavat tiedostot puretaan lopuksi kahteen hakemistoon - toiseen potentiaaliset entiteetit ja toiseen suhteet. 


In [1]:
# Voit luoda hakemistoja myös Google Colabroratoryyn ja ne säilyvät ympäristön käyttöajan
import os  

def make_directory(path):
  try:  
    os.mkdir(path)
  except OSError as error:  
    print(error)   
    
path = "/content"
# Luo ainakin hakemisto (UnzippedCDM_entities), johon kokoat valitut entiteetti-tiedostot 
# (cdm.json-loppuiset, joihin ei kuulu versionumerolliset ja manifest.cdm.json-loppuiset)
folder = input("Anna hakemiston nimi, kuten UnzippedCDM, johon haluat haluamasi json-loppuiset tiedostot: \n")
make_directory(os.path.join(path,folder))
# Listaa hakemistot   
!ls "/content" 

Anna hakemiston nimi, kuten UnzippedCDM, johon haluat haluamasi json-loppuiset tiedostot: 
ExampleCDM
ExampleCDM  sample_data


In [2]:
!wget https://github.com/microsoft/CDM/archive/master.zip -O temp.zip

--2020-12-17 16:20:11--  https://github.com/microsoft/CDM/archive/master.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/microsoft/CDM/zip/master [following]
--2020-12-17 16:20:11--  https://codeload.github.com/microsoft/CDM/zip/master
Resolving codeload.github.com (codeload.github.com)... 13.112.159.149
Connecting to codeload.github.com (codeload.github.com)|13.112.159.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘temp.zip’

temp.zip                [<=>                 ]  78.42M  8.88MB/s    in 14s     

2020-12-17 16:20:26 (5.67 MB/s) - ‘temp.zip’ saved [82228519]



In [3]:
# Lasketaan kaikkien tiedostojen lkm
from zipfile import ZipFile
import re

with ZipFile('/content/temp.zip', 'r') as zipObj:
   FileNames = zipObj.namelist()
   file_counter = 0
   for fileName in FileNames:
       file_counter = file_counter+1
print(file_counter)

53930


In [ ]:
# Lasketaan kaikkien tiedostojen lkm, jotka loppuvat cdm.json ja joissa ei ole versionumeroa, 
# ja jotka ovat schemaDocuments-hakemistossa, koska sivulla https://github.com/microsoft/CDM oleva 
# Entity Reference Index -linkki johtaa sivustolle https://github.com/microsoft/CDM/tree/master/schemaDocuments#directory-of-cdm-entities
# Jos tiedoston polkunimessä on Tables, niin silloin kyseessä on fyysiset entiteetit https://github.com/microsoft/CDM/blob/master/schemaDocuments/core/operationsCommon/Tables/README.md
# Jos tiedoston polkunimessä on Entities, niin silloin kyseessä on loogiset entiteetit https://github.com/microsoft/CDM/tree/master/schemaDocuments/core/operationsCommon/Entities
# Jos tiedoston polkunimessä on applicationCommon, niin silloin kyseessä on CDM:n vakio (standard) entiteetit https://github.com/microsoft/CDM/blob/master/schemaDocuments/core/applicationCommon/README.md
from zipfile import ZipFile
import re
with ZipFile('/content/temp.zip', 'r') as zipObj:
   FileNames = zipObj.namelist()
   # Käy tiedostonimet läpi
   file_counter = 0
   file_counter_manifest = 0
   file_counter_schema = 0
   file_counter_schema_manifest = 0
   file_counter_tables = 0
   file_counter_schema_tables = 0
   file_counter_entities = 0
   file_counter_schema_entities = 0
   file_counter_standard = 0
   file_counter_schema_standard = 0
   for fileName in FileNames:
       endswith = ".cdm.json"
       if fileName.endswith(endswith):
          pattern_1 = ".1."
          matchObj_1 =  re.search(pattern_1, fileName)
          pattern_2 = ".0."
          matchObj_2 =  re.search(pattern_2, fileName)
          pattern_3 = ".manifest.cdm.json"
          matchObj_3 =  re.search(pattern_3, fileName)
          pattern_4 = "schemaDocuments"
          matchObj_4 =  re.search(pattern_4, fileName)
          pattern_5 = "Tables"
          matchObj_5 =  re.search(pattern_5, fileName)
          pattern_6 = "Entities"
          matchObj_6 =  re.search(pattern_6, fileName)
          pattern_7 = "applicationCommon"
          matchObj_7 =  re.search(pattern_7, fileName)
          if not matchObj_1 and not matchObj_2:
             file_counter = file_counter+1
             if matchObj_3:
               file_counter_manifest = file_counter_manifest + 1
             if matchObj_4:
               file_counter_schema = file_counter_schema + 1 
             if matchObj_3 and matchObj_4:
               file_counter_schema_manifest = file_counter_schema_manifest + 1
             if matchObj_5:
               file_counter_tables = file_counter_tables + 1
             if matchObj_5 and matchObj_4 and matchObj_3:
               file_counter_schema_tables = file_counter_schema_tables + 1
             if matchObj_6:
               file_counter_entities = file_counter_entities + 1
             if matchObj_6 and matchObj_4 and matchObj_3:
               file_counter_schema_entities = file_counter_schema_entities + 1
             if matchObj_7:
               file_counter_standard = file_counter_standard + 1
             if matchObj_7 and matchObj_4 and matchObj_3:
               file_counter_schema_standard = file_counter_schema_standard + 1

print("Kaikki cdm.json-loppuiset tiedostot yhteensä:", file_counter)
print("Kaikki manifest.cdm.json-loppuiset tiedostot yhteensä:", file_counter_manifest)
print("Kaikki cdm.json-tiedostot, jotka eivät ole manifest.cdm.json-loppuisia:", file_counter - file_counter_manifest)
print("Kaikki cdm.json-loppuiset tiedostot schemaDocuments-hakemistossa yhteensä:", file_counter_schema)
print("Kaikki manifest.cdm.json-loppuiset tiedostot schemaDocuments-hakemistossa yhteensä:", file_counter_schema_manifest)
print("Kaikki cdm.json-tiedostot, jotka eivät ole manifest.cdm.json-loppuisia, schemaDocuments-hakemistossa:", file_counter_schema - file_counter_schema_manifest)
print("Kaikki cdm.json-tiedostot, joiden nimessä on Tables:", file_counter_tables)
print("Kaikki cdm.json-tiedostot, joiden nimessä on Tables ja manifest.cdm.json, schemaDocuments-hakemistossa:", file_counter_schema_tables)
print("Kaikki cdm.json-tiedostot, joiden nimessä on Entities:", file_counter_entities)
print("Kaikki cdm.json-tiedostot, joiden nimessä on Entities ja manifest.cdm.json, schemaDocuments-hakemistossa:", file_counter_schema_entities)
print("Kaikki cdm.json-tiedostot, joiden nimessä on applicationCommon:", file_counter_standard)
print("Kaikki cdm.json-tiedostot, joiden nimessä on applicationCommon ja manifest.cdm.json, schemaDocuments-hakemistossa:", file_counter_schema_standard)

Kaikki cdm.json-loppuiset tiedostot yhteensä: 18523
Kaikki manifest.cdm.json-loppuiset tiedostot yhteensä: 1580
Kaikki cdm.json-tiedostot, jotka eivät ole manifest.cdm.json-loppuisia: 16943
Kaikki cdm.json-loppuiset tiedostot schemaDocuments-hakemistossa yhteensä: 10536
Kaikki manifest.cdm.json-loppuiset tiedostot schemaDocuments-hakemistossa yhteensä: 579
Kaikki cdm.json-tiedostot, jotka eivät ole manifest.cdm.json-loppuisia, schemaDocuments-hakemistossa: 9957
Kaikki cdm.json-tiedostot, joiden nimessä on Tables: 6468
Kaikki cdm.json-tiedostot, joiden nimessä on Tables ja manifest.cdm.json, schemaDocuments-hakemistossa: 467
Kaikki cdm.json-tiedostot, joiden nimessä on Entities: 2975
Kaikki cdm.json-tiedostot, joiden nimessä on Entities ja manifest.cdm.json, schemaDocuments-hakemistossa: 60
Kaikki cdm.json-tiedostot, joiden nimessä on applicationCommon: 1395
Kaikki cdm.json-tiedostot, joiden nimessä on applicationCommon ja manifest.cdm.json, schemaDocuments-hakemistossa: 39


In [6]:
# Puretaan yhteen hakemistoon cdm.json-tiedostot halutusta schemaDocuments-hakemistosta
import re
from zipfile import ZipFile
listOfFileNames = {}
infiles = []
# Hakemisto, johon puretaan halutut tiedostot
folder = "/content/ExampleCDM"
with ZipFile('/content/temp.zip', 'r') as zipObj:
   listOfFileNames = zipObj.namelist()
   # Käy tiedostonimet läpi
   for fileName in listOfFileNames:
       # Tarkistaa ja purkaa tiedostot, jotka päättyvät halutulla tavalla
       endswith = ".cdm.json"
       if fileName.endswith(endswith):
          if fileName.endswith(endswith):
            pattern_1 = ".1."
            matchObj_1 =  re.search(pattern_1, fileName)
            pattern_2 = ".0."
            matchObj_2 =  re.search(pattern_2, fileName)
            pattern_3 = ".manifest.cdm.json"
            matchObj_3 =  re.search(pattern_3, fileName)
            pattern_4 = "schemaDocuments"
            matchObj_4 =  re.search(pattern_4, fileName)
            pattern_5 = "Table"
            matchObj_5 =  re.search(pattern_5, fileName)
            pattern_6 = "Entities"
            matchObj_6 =  re.search(pattern_6, fileName)
            pattern_7 = "example-public-standards"
            matchObj_7 =  re.search(pattern_7, fileName)
            if matchObj_7 and not matchObj_2 and not matchObj_1 :
              target_file = fileName.replace('/', '_')
              with open(folder+'/'+target_file, "wb") as f:
                  f.write(zipObj.read(fileName))

In [7]:
# Lasketaan tiedostojen lukumäärä hakemistoissa, johon schemaDocuments-hakemiston cdm.json-tiedostot on purettu
from pathlib import Path
import os

def list_files_from_folders(path,files):
    for element in path.iterdir():
      if element.is_dir():
         list_files_from_folders(element,files)
      if element.is_file():
         files.append(element) 

unzipped_CDM_folder = "/content/ExampleCDM"
os.chdir(unzipped_CDM_folder)
Path = Path.cwd()
files=[]
list_files_from_folders(Path,files)
print(len(files))    

204


In [ ]:
!zip -r /content/files.zip /content/TableCDM

In [ ]:
#ladataan Colabiin analysoitava tiedosto
from google.colab import files
downloaded = files.download('/content/files.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Jos ja kun olet mountannut driven, niin avaa drive ja drag-and-drop kansio, jossa on puretut json-tiedostot My Driveen 
# My Drivestä voit ladata zip-pakettina tiedostot työasemallesi ja edelleen OneDriveen